In [8]:
import pandas as pd

orders = pd.read_csv('orders.csv')
print("Orders Data:")
print(orders.head())
print(orders.info())

Orders Data:
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  objec

In [9]:
users = pd.read_json('users.json')
print("\nUsers Data:")
print(users.head())
print(users.info())


Users Data:
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 58.7+ KB
None


In [10]:
restaurants_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        if line.startswith('INSERT INTO'):
            match = re.search(r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([\d.]+)\)", line)
            if match:
                restaurants_data.append(match.groups())

In [11]:
restaurants = pd.DataFrame(restaurants_data, columns=['restaurant_id', 'restaurant_name_sql', 'cuisine', 'rating'])
restaurants['restaurant_id'] = pd.to_numeric(restaurants['restaurant_id'])
restaurants['rating'] = pd.to_numeric(restaurants['rating'])

In [12]:
print("\nRestaurants Data (from SQL):")
print(restaurants.head())
print(restaurants.info())


Restaurants Data (from SQL):
   restaurant_id restaurant_name_sql  cuisine  rating
0              1        Restaurant_1  Chinese     4.8
1              2        Restaurant_2   Indian     4.1
2              3        Restaurant_3  Mexican     4.3
3              4        Restaurant_4  Chinese     4.1
4              5        Restaurant_5  Chinese     4.8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   restaurant_id        500 non-null    int64  
 1   restaurant_name_sql  500 non-null    object 
 2   cuisine              500 non-null    object 
 3   rating               500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 11.8+ KB
None


In [13]:
merged_1 = pd.merge(orders, users, on='user_id', how='left')

In [14]:
final_df = pd.merge(merged_1, restaurants, on='restaurant_id', how='left')

In [15]:
print("\nFinal Merged Data:")
print(final_df.head())
print(final_df.info())


Final Merged Data:
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_sql  cuisine  rating  
0      Restaurant_450  Mexican     3.2  
1      Restaurant_309   Indian     4.5  
2      Restaurant_107  Mexican     4.0  
3   

In [16]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

In [17]:
df = pd.read_csv('final_food_delivery_dataset.csv')

In [18]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

In [41]:
q1 = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
q1_val = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().max()
print(f" {q1} ({q1_val})")

 Chennai (1080909.79)


In [24]:
q2 = df.groupby('cuisine')['total_amount'].mean().idxmax()
q2_val = df.groupby('cuisine')['total_amount'].mean().max()
print(f" {q2} ({q2_val})")

 Mexican (808.0213444401395)


In [25]:
user_totals = df.groupby('user_id')['total_amount'].sum()
q3_count = user_totals[user_totals > 1000].count()
print(f" {q3_count}")

 2544


In [26]:
bins = [2.9, 3.5, 4.0, 4.5, 5.1]
labels = ['3.0 - 3.5', '3.6 - 4.0', '4.1 - 4.5', '4.6 - 5.0']
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels)
q4 = df.groupby('rating_range')['total_amount'].sum().idxmax()
q4_val = df.groupby('rating_range')['total_amount'].sum().max()
print(f" {q4} ({q4_val})")

<ipython-input-26-05360c7fa409>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q4 = df.groupby('rating_range')['total_amount'].sum().idxmax()
<ipython-input-26-05360c7fa409>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q4_val = df.groupby('rating_range')['total_amount'].sum().max()


 4.6 - 5.0 (2197030.75)


In [27]:
q5 = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()
q5_val = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().max()
print(f" {q5} ({q5_val})")

 Chennai (808.4590800299178)


In [29]:
cuisine_stats = df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
)
print(f"\n{cuisine_stats}")


         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09


In [30]:
gold_count = df[df['membership'] == 'Gold'].shape[0]
total_count = df.shape[0]
q7_perc = round((gold_count / total_count) * 100)
print(f" {q7_perc}%")

 50%


In [31]:
rest_stats = df.groupby('restaurant_name').agg(
    order_count=('order_id', 'count'),
    aov=('total_amount', 'mean')
)
q8_candidates = rest_stats[rest_stats['order_count'] < 20]
q8 = q8_candidates['aov'].idxmax()
q8_val = q8_candidates['aov'].max()
print(f" {q8} (AOV: {q8_val}, Orders: {q8_candidates.loc[q8, 'order_count']})")

 Hotel Dhaba Multicuisine (AOV: 1040.2223076923076, Orders: 13)


In [32]:
q9_combos = df.groupby(['membership', 'cuisine'])['total_amount'].sum()
q9 = q9_combos.idxmax()
q9_val = q9_combos.max()
print(f" {q9} ({q9_val})")

 ('Regular', 'Mexican') (1072943.3)


In [33]:
df['quarter'] = df['order_date'].dt.quarter
q10 = df.groupby('quarter')['total_amount'].sum().idxmax()
q10_val = df.groupby('quarter')['total_amount'].sum().max()
print(f" Q{q10} ({q10_val})")

 Q3 (2037385.1)


In [35]:
gold_orders_count = df[df['membership'] == 'Gold'].shape[0]
print(f" {gold_orders_count}")

 4987


In [36]:
hyderabad_revenue = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())
print(f" {hyderabad_revenue}")

 1889367


In [37]:
distinct_users_count = df['user_id'].nunique()
print(f" {distinct_users_count}")

 2883


In [38]:
gold_aov = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)
print(f" {gold_aov}")

 797.15


In [39]:
high_rating_orders_count = df[df['rating'] >= 4.5].shape[0]
print(f" {high_rating_orders_count}")

 3374


In [42]:
gold_df = df[df['membership'] == 'Gold']
top_gold_city = gold_df.groupby('city')['total_amount'].sum().idxmax()
top_gold_city_orders_count = gold_df[gold_df['city'] == top_gold_city].shape[0]
print(f" {top_gold_city}")

 Chennai
